# Data Cleaning

This notebook uses the scraped data to fill in missing values in the dataset. The output is a csv file that is a combination of reviews and product features. These files are consumed by the model.

In [112]:
import pandas as pd
import gzip
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [113]:
# configure file names
reviews_file = 'reviews-women-full.csv'
output_file = 'reviews_Women_all_final' + '_scraped.csv'

In [114]:
# Reading Julian's files
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [115]:
# read reviews file
# reviews = getDF(reviews_file)
reviews = pd.read_csv(reviews_file, names=['reviewerId', 'asin', 'unixtime'])
print reviews.columns
print reviews.shape

Index([u'reviewerId', u'asin', u'unixtime'], dtype='object')
(2855258, 3)


In [116]:
# read product metadata
meta = pd.read_hdf('womens_Meta_scraped.hdf5', key='data')
print meta.shape
print meta.columns

(603277, 16)
Index([u'asin', u'categories', u'title', u'price', u'salesRank', u'imUrl',
       u'brand', u'related', u'description', u'BRAND', u'FEATURE_BULLETS',
       u'NAME', u'ORIGINAL_PRICE', u'PRODUCT_DESCRIPTION',
       u'PRODUCT_INFORMATION', u'SALE_PRICE'],
      dtype='object')


## Clean Up Price, Brand, Description

### Price

In [117]:
import re

# the following functions are used to get the final value of the price from the scraped data

def is_number(s):
    try:
        complex(s) # for int, long, float and complex
    except ValueError:
        return False

    return True
getMoney = lambda text : [ ''.join(x[0]) for x in re.findall(r'(\$?\d+([,\.]\d+)?k?)', text.replace(',', '').replace('$',''))]

def getAvg(x):
    if np.isnan(x.price) and (pd.isnull(x.SALE_PRICE) == False or 
                              pd.isnull(x.ORIGINAL_PRICE) == False):
        # each of these could be a high/low range
        s = None
        o = None
        if not pd.isnull(x.SALE_PRICE): 
            s = average([(float(y) if is_number(y) else -1) for y in getMoney(x.SALE_PRICE)])
        if not pd.isnull(x.ORIGINAL_PRICE):
            o = average([(float(y) if is_number(y) else -1) for y in getMoney(x.ORIGINAL_PRICE)])
        return average([s if not pd.isnull(x.SALE_PRICE) else o, o if not pd.isnull(x.ORIGINAL_PRICE) else s])
    else:
        return x.price

In [118]:
print 'length:', meta.shape[0]
print 'missing price:', meta[(meta.price.isnull()) | (meta.price == -1)].shape[0]

length: 603277
missing price: 349160


In [119]:
# fill in missing price with scraped price
meta[['price']] = meta.apply(lambda x: getAvg(x), axis='columns')
print 'missing price', meta[(meta.price.isnull()) | (meta.price == -1)].shape[0]

missing price 278835


In [120]:
meta[(meta.price.isnull()) ].head()

,asin,categories,title,price,salesRank,imUrl,brand,related,description,BRAND,FEATURE_BULLETS,NAME,ORIGINAL_PRICE,PRODUCT_DESCRIPTION,PRODUCT_INFORMATION,SALE_PRICE
0,0456844570,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",RiZ Women's Beautify Crafted &frac12; Rimmed F...,NaN,{u'Clothing': 1180499},http://ecx.images-amazon.com/images/I/31QZTHxv...,NaN,"{u'also_viewed': [u'B008MTRT1O', u'B00BUG47S4'...",NaN,Riz,"[Plastic frame, Plastic lens, non-polarized, L...",RiZ Women's Beautify Crafted ½ Rimmed Floral C...,None,The perfect accessory for that summer dress. 4...,{},None
1,0641674791,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",Red Crocodile Embossed Leather Travel Wallet,NaN,NaN,http://ecx.images-amazon.com/images/I/31wW2pPm...,NaN,NaN,ISBN: 0641674791\nISBN-13: 9780641674792\nManu...,Punctuate,"[This genuine leather travel wallet, covered i...",Red Crocodile Embossed Leather Travel Wallet,None,Product Description\nISBN: 0641674791\nISBN-13...,{},None
2,0681358173,"[[Clothing, Shoes & Jewelry, Women, Petite, Fa...",Bear Ear Hoodie Jacket &amp; Poms on drawstrin...,NaN,{u'Toys & Games': 600198},http://ecx.images-amazon.com/images/I/51JCLkQk...,NaN,NaN,Adorable fleece teddy bear ear sweater for adu...,Tasman,"[Note: Korean Fashionwear sizes run smaller, S...",Bear Ear Hoodie Jacket & Poms on drawstrings ~...,None,Product Description\nAdorable fleece teddy bea...,{},None
4,1304743594,"[[Clothing, Shoes & Jewelry, Women, Watches, W...",Watch Black Match Iphone,NaN,NaN,http://ecx.images-amazon.com/images/I/4135gTwj...,NaN,"{u'also_viewed': [u'1300827327', u'1304743128'...",NaN,alexgurman.com,[],Watch Black Match Iphone,None,,{},None
5,1394751931,"[[Clothing, Shoes & Jewelry, Women, Watches, W...",Luxury Dark Angel's Heart Women Crystal Bracel...,NaN,NaN,http://ecx.images-amazon.com/images/I/51wCNVS%...,NaN,NaN,NaN,new brand,[],Luxury Dark Angel's Heart Women Crystal Bracel...,None,,{},None


### Brand

In [121]:
print 'length:', meta.shape[0]
print 'missing brand:', meta[(meta.brand == '') | (meta.brand.isnull()) | (meta.brand.str.contains('nknown'))].shape[0]

length: 603277
missing brand: 579363


In [122]:
brand = lambda x: ' '.join([y.strip() for y in unicode(x.BRAND if pd.isnull(x.brand) or str(x.brand).find('nknown') >= 0 or str(x.brand) == '' else x.brand).split(',')])
meta.brand = meta.apply(lambda x: brand(x), axis='columns')
print 'missing brand:', meta[(meta.brand == '') | (meta.brand.isnull()) | (meta.brand.str.contains('nknown'))].shape[0]

missing brand: 1084


In [123]:
meta[(meta.brand == '') | (meta.brand.isnull()) | (meta.brand.str.contains('nknown'))][['brand', 'BRAND']].drop_duplicates()

,brand,BRAND
183,Unknown,Unknown
168127,nknown,nknown


### Description

In [124]:
print 'length:', meta.shape[0]
print 'missing description:', meta[(meta.description == '') | (meta.description.isnull())].shape[0]

length: 603277
missing description: 587975


In [125]:
description = lambda x: ' '.join([y.strip() for y in x.PRODUCT_DESCRIPTION.split(',')]) if pd.isnull(x.description) or x.description.strip() == '' else x.description
meta.description = meta.apply(lambda x: description(x), axis='columns')
print 'missing description:', meta[(meta.description == '') | (meta.description.isnull())].shape[0]

missing description: 82541


### Create output file

Desired Output columns and order:
```python
[u'reviewerId', u'asin', u'unixReviewTime', u'brand', u'price']
```

In [126]:
reviews.columns

Index([u'reviewerId', u'asin', u'unixtime'], dtype='object')

In [127]:
meta.columns

Index([u'asin', u'categories', u'title', u'price', u'salesRank', u'imUrl',
       u'brand', u'related', u'description', u'BRAND', u'FEATURE_BULLETS',
       u'NAME', u'ORIGINAL_PRICE', u'PRODUCT_DESCRIPTION',
       u'PRODUCT_INFORMATION', u'SALE_PRICE'],
      dtype='object')

In [128]:
output = pd.merge(reviews[[c for c in reviews.columns if c not in ['brand', 'price']]], 
                  meta, 
                  on='asin', 
                  how='left')[[u'reviewerId', u'asin', u'unixtime', u'brand', u'price', u'description']]

In [ ]:
output.to_csv(output_file, index=False, encoding='utf-8')